### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    ftm = str(today - relativedelta(days=1))
    itm = str((today - relativedelta(days=1)).replace(day=1))
else:
    itm = str(today.replace(day=1))
    ftm = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))

### GOOGLE SHEETS

In [4]:
# Partners PEYA
sheet_id = '1HmAvHYbJJa3JyRTgRJGKXataQB_TvFwNzyo8a6qXd2o'
wks_name = 'Partners PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
partners = sheet.sheet_to_df(index=0,header_rows=1)

In [5]:
# Stats PEYA
sheet_id = '1PO5kGLgz5Xn_WNCQQB0OPcKCl7KeS7kMrkJaGxafqmY'
wks_name = 'Stats PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
stats = sheet.sheet_to_df(index=0,header_rows=1)

### TRABAJO

In [102]:
# Filtro las KAMS
kams = partners[partners['KAM'] == 'Si'].copy()
# Filtro las columnas a usar
cols = ['Account_Owner','Grid','Id','Franchise','Name','City','Area','Main_Cuisine','Logistic','Online','Online_TM']
# Preparo KAMs TM y LM
kams_tm = kams[cols].copy()
kams_lm = kams[cols].copy()
# Cambio nombre a las columnas para evitar errores con Logistic y las órdenes Logísticas
cols = ['Account_Owner','Grid','Id','Franchise','Name','City','Area','Main_Cuisine','Is Logistic','Online','Online_TM']
kams_tm.columns = cols
kams_lm.columns = cols

In [103]:
# Separo las columnas según TM y LM
cols = ['Active_Users','Confirmed_Voucher','Confirmed','Logistic','Confirmed_OP','VFR_Num','Total',
       'DF_No_Discount','Sessions','Transactions','GMV','Revenue','VL_Denominador','VL_Numerador']
cols_tm = ['Id']+[i+' TM' for i in cols]
cols_lm = ['Id']+[i+' LM' for i in cols]

In [104]:
# Separo las Stats TM y LM
stats_tm = stats[cols_tm].copy()
stats_lm = stats[cols_lm].copy()
# Doy formato a las columnas
stats_tm = stats_tm.astype(float)
stats_lm = stats_lm.astype(float)

In [105]:
# Doy formato a las columnas
kams_tm['Id'] = kams_tm['Id'].astype(float)
kams_lm['Id'] = kams_lm['Id'].astype(float)
# Uno las tablas
kams_tm = kams_tm.merge(stats_tm,on=['Id'],how='left')
kams_lm = kams_lm.merge(stats_lm,on=['Id'],how='left')

In [106]:
# Borro los TM y LM de las columnas
cols_tm = [i.replace(' TM','') for i in kams_tm.columns]
cols_lm = [i.replace(' LM','') for i in kams_lm.columns]
# Reemplazo los nombres de las columnas
kams_tm.columns = cols_tm
kams_lm.columns = cols_lm

In [107]:
# Creo las columnas faltantes
kams_tm['VFR'] = kams_tm['VFR_Num'] / kams_tm['Total']
kams_lm['VFR'] = kams_lm['VFR_Num'] / kams_lm['Total']
kams_tm['CVR'] = kams_tm['Transactions'] / kams_tm['Sessions']
kams_lm['CVR'] = kams_lm['Transactions'] / kams_lm['Sessions']
kams_tm.replace([np.nan,np.inf,-np.inf],0,inplace=True)
kams_lm.replace([np.nan,np.inf,-np.inf],0,inplace=True)
# Cambio el CVR sin Sesiones a '-'
kams_tm['CVR'] = kams_tm['CVR'].apply(lambda x: '-' if x == 0 else x)
kams_lm['CVR'] = kams_lm['CVR'].apply(lambda x: '-' if x == 0 else x)

### CARGA

In [100]:
# Carga TM
sheet_id = '1K_H2gVUHQ0NA9eoCjrAxwgigl_HBJz-PUcbGEdrSMuY'
wks_name = 'TM'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(kams_tm, index=False, sheet=wks_name, replace=True)

{'spreadsheetId': '1K_H2gVUHQ0NA9eoCjrAxwgigl_HBJz-PUcbGEdrSMuY',
 'replies': [{'findReplace': {'valuesChanged': 81557,
    'rowsChanged': 4921,
    'sheetsChanged': 1,
    'occurrencesChanged': 81557}}]}

In [108]:
# Carga LM
sheet_id = '1K_H2gVUHQ0NA9eoCjrAxwgigl_HBJz-PUcbGEdrSMuY'
wks_name = 'LM'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(kams_lm, index=False, sheet=wks_name, replace=True)

{'spreadsheetId': '1K_H2gVUHQ0NA9eoCjrAxwgigl_HBJz-PUcbGEdrSMuY',
 'replies': [{'findReplace': {'valuesChanged': 81536,
    'rowsChanged': 4921,
    'sheetsChanged': 1,
    'occurrencesChanged': 81536}}]}